In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ISORegressorCNN(nn.Module):
    def __init__(self):
        super(ISORegressorCNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)

        # Pooling layers
        self.pool = nn.MaxPool2d(2, 2)

        # Fully connected layers for regression
        self.fc1 = nn.Linear(256 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 1)  # Final output for ISO prediction

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = F.relu(self.conv4(x))
        x = self.pool(x)

        # Flatten the output
        x = x.view(-1, 256 * 16 * 16)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # Directly output ISO value

        return x


In [7]:
from torchsummary import summary

# Instantiate your model
model = ISORegressorCNN().cuda()

# Print a summary
summary(model, (3, 256, 256), device='cuda')  

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 256, 256]             896
         MaxPool2d-2         [-1, 32, 128, 128]               0
            Conv2d-3         [-1, 64, 128, 128]          18,496
         MaxPool2d-4           [-1, 64, 64, 64]               0
            Conv2d-5          [-1, 128, 64, 64]          73,856
         MaxPool2d-6          [-1, 128, 32, 32]               0
            Conv2d-7          [-1, 256, 32, 32]         295,168
         MaxPool2d-8          [-1, 256, 16, 16]               0
            Linear-9                  [-1, 512]      33,554,944
           Linear-10                  [-1, 128]          65,664
           Linear-11                    [-1, 1]             129
Total params: 34,009,153
Trainable params: 34,009,153
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.75
Fo

In [13]:
from torchviz import make_dot

dummy_input = torch.randn(1, 3, 256, 256).cuda()

# Forward pass to get output
output = model(dummy_input)

# Visualize the model
make_dot(output, params=dict(model.named_parameters())).render("ISORegressorCNN", format="png")


'ISORegressorCNN.png'